<a href="https://colab.research.google.com/github/SylviaNice/7162856-G-rez-Git-et-GitHub/blob/main/Projet_BottleNeck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
# 1. Nettoyage de l'environnement (efface toutes les variables en mémoire)

%reset -f

In [144]:
# 2. Installation des bibliothèques

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

In [145]:
# 3. Importation des données

from google.colab import files
uploaded = files.upload()

Saving fichier_liaison.xlsx to fichier_liaison (4).xlsx
Saving Fichier_erp.xlsx to Fichier_erp (4).xlsx
Saving Fichier_web.xlsx to Fichier_web (4).xlsx


In [146]:
# 4. Chargement des 3 fichiers

erp_file = "Fichier_erp.xlsx"
web_file = "Fichier_web.xlsx"
liaison_file = "fichier_liaison.xlsx"

In [147]:
# 5. Lecture des fichiers

erp_df = pd.read_excel(erp_file)
web_df = pd.read_excel(web_file)
liaison_df = pd.read_excel(liaison_file)

In [148]:
# APERÇU DU FICHIER ERP

display(erp_df.head())
print(erp_df.shape)
print(erp_df.dtypes)

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


(825, 5)
product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
dtype: object


In [149]:
# NETTOYAGE DU FICHIER ERP

# Afficher toutes les incohérences dans les stocks fichier erp
stocks_negatifs = erp_df[erp_df['stock_quantity'] < 0]
print("Stocks négatifs :")
print(stocks_negatifs)

Stocks négatifs :
     product_id  onsale_web  price  stock_quantity stock_status
449        4973           0   10.0              -1   outofstock
573        5700           1   44.5              -1   outofstock


In [150]:
# Corriger les stocks négatifs
erp_df['stock_quantity'] = erp_df['stock_quantity'].abs()
erp_df['stock_quantity'] = erp_df['stock_quantity'].clip(lower=0)

In [151]:
# Détecter les colonnes avec des valeurs manquantes
missing_columns = erp_df.columns[erp_df.isna().any()].tolist()
print("Colonnes avec des valeurs manquantes :", missing_columns)

Colonnes avec des valeurs manquantes : []


In [152]:
# Détecter les doublons
doublons_erp = erp_df.duplicated(keep=False)
print("Nombre de doublons :", doublons_erp.sum())

Nombre de doublons : 0


In [153]:
# Supprimer les lignes entièrement vides du fichier erp

erp_df.dropna(how='all', inplace=True)
print(erp_df.shape)

(825, 5)


In [154]:
# Enregistrement du fichier nettoyé erp

erp_df.to_excel('Fichier_erp_clean.xlsx', index=False)
files.download("Fichier_erp_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [155]:
# APERÇU DU FICHIER LIAISON

display(liaison_df.head())
print(liaison_df.shape)
print(liaison_df.dtypes)

,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


(825, 2)
product_id     int64
id_web        object
dtype: object


In [156]:
# Corriger l'erreur dans le nom de la colonne

liaison_df = liaison_df.rename(columns={'id_web': 'sku'})
print(liaison_df.columns)

Index(['product_id', 'sku'], dtype='object')


In [157]:
# 23. Visualiser la cellule erreur "bon-cadeau-25-euros"

print(liaison_df.loc[440:449, ['sku']])

                     sku
440                13379
441                15338
442                15337
443  bon-cadeau-25-euros
444                15737
445                15958
446                16515
447                16586
448                11225
449                  NaN


In [158]:
# 24. Remplacer la cellule "bon-cadeau-25-euros" par une cellule vide

liaison_df.loc[443,  'sku'] = ""
print(liaison_df.loc[440:449, ['sku']])

       sku
440  13379
441  15338
442  15337
443       
444  15737
445  15958
446  16515
447  16586
448  11225
449    NaN


In [159]:
# Supprimer les indices -1 dans la colonne 'sku' du fichier final

liaison_df['sku'] = liaison_df['sku'].apply(lambda x: x.strip().replace(r'-1', '') if isinstance(x, str) else x )
print(liaison_df.loc[822:825, ['sku']])

       sku
822  13127
823  14680
824  16230


In [160]:
# Enregistrement du fichier nettoyé liaison

liaison_df.to_excel('fichier_liaison_clean.xlsx', index=False)
files.download("fichier_liaison_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [161]:
# APERÇU DU FICHIER web

display(web_df.head())
print(web_df.shape)
print(web_df.dtypes)

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,16004,0,0,0,0.0,5.0,NaN,NaN,2.0,2018-06-07 16:27:25,2018-06-07 14:27:25,NaN,Château du Couvent Pomerol 2017,NaN,publish,closed,closed,NaN,chateau-du-couvent-pomerol-2017,2020-08-25 18:35:02,2020-08-25 16:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15075,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-14 15:39:43,2018-02-14 14:39:43,NaN,Parés Baltà Penedès Indigena 2017,Des couleurs et aromes intenses où le fruit et...,publish,closed,closed,NaN,pares-balta-penedes-indigena-2017,2020-08-20 15:35:02,2020-08-20 13:35:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,16209,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-14 17:15:31,2018-02-14 16:15:31,NaN,Maurel Cabardès Tradition 2017,"Un joli nez aux arômes de fruits rouges, de ca...",publish,closed,closed,NaN,maurel-cabardes-tradition-2017,2020-08-05 18:05:03,2020-08-05 16:05:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,15763,0,0,0,0.0,1.0,NaN,NaN,2.0,2020-01-25 14:08:16,2020-01-25 13:08:16,NaN,Domaine de la Monardière Vacqueyras Vieilles V...,NaN,publish,closed,closed,NaN,domaine-de-la-monardiere-vacqueyras-les-vieill...,2020-08-21 11:35:02,2020-08-21 09:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


(1513, 28)
sku                              object
virtual                           int64
downloadable                      int64
rating_count                      int64
average_rating                  float64
total_sales                     float64
tax_status                       object
tax_class                       float64
post_author                     float64
post_date                datetime64[ns]
post_date_gmt            datetime64[ns]
post_content                    float64
post_title                       object
post_excerpt                     object
post_status                      object
comment_status                   object
ping_status                      object
post_password                   float64
post_name                        object
post_modified            datetime64[ns]
post_modified_gmt        datetime64[ns]
post_content_filtered           float64
post_parent                     float64
guid                             object
menu_order                   

In [162]:
# Supprimer les lignes entièrement vides

web_df.dropna(how='all', inplace=True)
print(web_df.shape)

(1513, 28)


In [165]:
# Supprimer les doublons sur la clé avant la fusion

web_df = web_df.drop_duplicates(subset=['sku'])
print(web_df.shape)

# Save the cleaned web_df to an Excel file
web_df.to_excel('Fichier_web_clean.xlsx', index=False)

files.download("Fichier_web_clean.xlsx")

(715, 28)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>